In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # @param ["tensorflow", "jax", "torch"]

import json
import math
import numpy as np

import keras
from keras import losses
from keras import ops
from keras import optimizers
from keras.optimizers import schedules
from keras import metrics

import keras_cv

# Import tensorflow for [`tf.data`](https://www.tensorflow.org/api_docs/python/tf/data) and its preprocessing functions
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
classifier = keras_cv.models.ImageClassifier.from_preset(
    "efficientnetv2_b0_imagenet_classifier"
)

In [ ]:
filepath = keras.utils.get_file(origin="https://i.imgur.com/9i63gLN.jpg")
image = keras.utils.load_img(filepath)
image = np.array(image)

In [ ]:
keras_cv.visualization.plot_image_gallery(
    np.array([image]), rows=1, cols=1, value_range=(0, 255), show=True, scale=4
)

In [ ]:
predictions = classifier.predict(np.expand_dims(image, axis=0)) # flatten 1d to 2d, list to matrix
# Reshaping data: To match the expected input shape of a model or function.
# Broadcasting: To enable broadcasting operations between arrays with different shapes.
# Tensor operations: To perform operations that require specific tensor shapes.

In [ ]:
top_classes = predictions[0].argsort(axis=-1) # argsort sort the array in terms of index

# import numpy as np

# arr = np.array([4, 2, 7, 1, 3])
# indices = np.argsort(arr)
# print(indices)  # [3 1 4 0 2]

Fine tuning a pretrained classifier

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
AUTOTUNE = tf.data.AUTOTUNE
tfds.disable_progress_bar()

In [ ]:
data, dataset_info = tfds.load("cats_vs_dogs", with_info=True, as_supervised=True)


In [ ]:
train_steps_per_epoch = dataset_info.splits["train"].num_examples // BATCH_SIZE

In [ ]:
train_dataset = data["train"]

In [ ]:
num_classes = dataset_info.features["label"].num_classes

In [ ]:
resizing = keras_cv.layers.Resizing(
    IMAGE_SIZE[0], IMAGE_SIZE[1], crop_to_aspect_ratio=True
)

In [ ]:
def preprocess_inputs(image, label):
    image = tf.cast(image, tf.float32)
    # Staticly resize images as we only iterate the dataset once.
    return resizing(image), tf.one_hot(label, num_classes)


In [ ]:
# Shuffle the dataset to increase diversity of batches.
# 10*BATCH_SIZE follows the assumption that bigger machines can handle bigger
# shuffle buffers.
train_dataset = train_dataset.shuffle(
    10 * BATCH_SIZE, reshuffle_each_iteration=True
).map(preprocess_inputs, num_parallel_calls=AUTOTUNE)

In [ ]:
train_dataset = train_dataset.batch(BATCH_SIZE)

In [ ]:
images = next(iter(train_dataset.take(1)))[0]
keras_cv.visualization.plot_image_gallery(images, value_range=(0, 255))

In [ ]:
model = keras_cv.models.ImageClassifier.from_preset(
    "efficientnetv2_b0_imagenet", num_classes=2
)
model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.SGD(learning_rate=0.01),
    metrics=["accuracy"],
)

In [ ]:
model.fit(train_dataset)

In [ ]:
predictions = model.predict(np.expand_dims(image, axis=0))

classes = {0: "cat", 1: "dog"}
print("Top class is:", classes[predictions[0].argmax()])